<a href="https://colab.research.google.com/github/alitarikarslanoglu/OntoGPT_FHC_EFSA/blob/faq/claim_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Claim Analysis

## System Check and Instalation of Reqirements

In [ ]:
  !nvidia-smi

Tue Apr  2 21:52:36 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### System Requirements

In [ ]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq torch==2.1.0 --progress-bar off
!pip install -qqq transformers==4.31.0 --progress-bar off
!pip install -qqq langchain==0.0.266 --progress-bar off
!pip install -qqq chromadb==0.4.5 --progress-bar off
!pip install -qqq pypdf==3.15.0 --progress-bar off
!pip install -qqq xformers==0.0.20 --progress-bar off
!pip install -qqq sentence_transformers==2.2.2 --progress-bar off
!pip install -qqq InstructorEmbedding==1.0.1 --progress-bar off
!pip install -qqq pdf2image==1.16.3 --progress-bar off
!pip install -qqq selenium==4.14.0 --progress-bar off
!pip install -qqq bs4 --progress-bar off
!pip install -qqq tqdm --progress-bar off
!pip install -qqq PyPDF2 --progress-bar off
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb && apt install ./google-chrome-stable_current_amd64.deb
!pip install -qqq  auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/ --progress-bar off
!apt install chromium-chromedriver
!sudo apt-get install poppler-utils

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.2.1+cu121 requires torch==2.2.1, but you have torch 2.1.0 which is incompatible.
torchtext 0.17.1 requires torch==2.2.1, but you have torch 2.1.0 which is incompatible.
torchvision 0.17.1+cu121 requires torch==2.2.1, but you have torch 2.1.0 which is incompatible.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take

### Imports

In [ ]:
#import
import torch
from tqdm import tqdm
from auto_gptq import AutoGPTQForCausalLM
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from pdf2image import convert_from_path
from transformers import AutoTokenizer, TextStreamer, pipeline
import pandas as pd
import re
import os
import datetime
import shutil
import glob
import time
from tqdm import tqdm
from google.colab import drive
from IPython.display import clear_output
from PyPDF2 import PdfReader
import random

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

## Retrival

### Selenium WebDriver for Chrome

In [ ]:
from selenium import webdriver

def driversetup():
    options = webdriver.ChromeOptions()
    #run Selenium in headless mode
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36")
    #overcome limited resource problems
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("lang=en")
    #open Browser in maximized mode
    options.add_argument("start-maximized")
    #disable infobars
    options.add_argument("disable-infobars")
    #disable extension
    options.add_argument('ignore-certificate-errors')
    options.add_argument('--ignore-ssl-errors=yes')
    options.add_argument("--disable-extensions")
    options.add_argument("--incognito")
    options.add_argument("--disable-blink-features=AutomationControlled")

    driver = webdriver.Chrome(options=options)

    return driver

###Expert Report/CSV Retrival

In [ ]:
!mkdir data
!mkdir pdfs
!gdown 1Xgb7GwakWa_wEiOAeiFz-sUg8Uo0NO3F -O data/tree.xlsx
!rm -r pdfs

Downloading...
From: https://drive.google.com/uc?id=1Xgb7GwakWa_wEiOAeiFz-sUg8Uo0NO3F
To: /content/data/tree.xlsx
100% 541k/541k [00:00<00:00, 84.9MB/s]


In [ ]:
claims = pd.read_excel('data/tree.xlsx')

### List of EFSA Opinion References Links

In the google sheet created by experts, "EFSA Opinion Reference" feature contains the codes of the documents. Following code creates the usable links to those Scientific opinions
https://efsa.onlinelibrary.wiley.com/doi/pdf/10.2903/j.efsa.2009.1226

In [ ]:
res = []
for k in tqdm(range(len(claims['EFSA Opinion Reference'])), position=0, leave=True):
  i = claims['EFSA Opinion Reference'][k]
  claim = claims['Claim'][k]
  if str(i) == 'nan':
    continue
  row = re.split(',',str(i))
  for j in row:
    args = re.split(';|:',j)
    if len(args)!=3:
      continue
    res.append('https://efsa.onlinelibrary.wiley.com/doi/pdf/10.2903/j.efsa.' + args[0].replace(" ","") + '.' + args[2].replace(" ","") + '#'+ claim + '#' + str(k))

  row = re.split('-',str(i))
  args = re.split('-',j)
  if len(args)!=3:
    continue
  res.append('https://efsa.onlinelibrary.wiley.com/doi/pdf/10.2903/j.efsa.' + args[1].replace(" ","") + '.' + args[2].replace(" ","") + '#'+ claim + '#' + str(k))

100%|██████████| 237/237 [00:00<00:00, 33455.05it/s]


In [ ]:
res[0]

'https://efsa.onlinelibrary.wiley.com/doi/pdf/10.2903/j.efsa.2009.1252#ALA contributes to the maintenance of normal blood cholesterol levels#1'

In [ ]:
def getpage(url, c, out_subdir):
  log = ''
  try:
    for k in os.listdir('pdfs/'+out_subdir+'/'):
      n = re.split('-',k)
      if len(n)!=8:
        continue
      if (str(c) + '.pdf') == n[7]:
        log += f'\tFile n {c} skipped already present\tfilename:{k}\n'
        return log
  except:
    pass

  driver = driversetup()
  driver.get(url)
  driver.close()

  name = 'pdfs/'+ out_subdir + '/efsa-' + str(datetime.datetime.now().strftime("%Y-%m-%d-at-%H-%M-")) + str(c) + '.pdf'

  try:
    filename = max([f for f in os.listdir('.')],key=os.path.getctime)
    try:
      os.mkdir('pdfs/'+ out_subdir)
    except:
      pass
    shutil.move(filename,os.path.join('.',name))
  except:
    log += f'File n {c} failed reffer to the followring url.\n{url}\n'
  return log

For each folders containing pdfs for claims, a text file created giving the overview of the claims

In [ ]:
!rm -r pdfs
!mkdir pdfs

In [ ]:
counter = 0

log = ''
for i in tqdm(res, position=0, leave=True):
  sep = i.split('#')
  log+=getpage(sep[0],counter, sep[2])
  with open('./pdfs/' + sep[2] + '/claim.txt', 'w') as f:
    f.write(sep[1])
  counter += 1

print(log)

100%|██████████| 297/297 [18:53<00:00,  3.82s/it]

File n 217 failed reffer to the followring url.
https://efsa.onlinelibrary.wiley.com/doi/pdf/10.2903/j.efsa.2010.4114
File n 218 failed reffer to the followring url.
https://efsa.onlinelibrary.wiley.com/doi/pdf/10.2903/j.efsa.2010.4114
File n 219 failed reffer to the followring url.
https://efsa.onlinelibrary.wiley.com/doi/pdf/10.2903/j.efsa.2010.4114
File n 222 failed reffer to the followring url.
https://efsa.onlinelibrary.wiley.com/doi/pdf/10.2903/j.efsa.2010.4114
File n 291 failed reffer to the followring url.
https://efsa.onlinelibrary.wiley.com/doi/pdf/10.2903/j.efsa.2013.00234
File n 292 failed reffer to the followring url.
https://efsa.onlinelibrary.wiley.com/doi/pdf/10.2903/j.efsa.2014.00073
File n 293 failed reffer to the followring url.
https://efsa.onlinelibrary.wiley.com/doi/pdf/10.2903/j.efsa.2013.00040
File n 294 failed reffer to the followring url.
https://efsa.onlinelibrary.wiley.com/doi/pdf/10.2903/j.efsa.2015.00437
File n 295 failed reffer to the followring url.
http

## Model

### Model Initialization

In [ ]:
model_path = "TheBloke/Llama-2-13B-chat-GPTQ"
model_basename = "model"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=True)
model = AutoGPTQForCausalLM.from_quantized(
    model_path,
    revision="gptq-4bit-128g-actorder_True",
    model_basename=model_basename,
    use_safetensors=True,
    trust_remote_code=True,
    inject_fused_attention=False,
    device=DEVICE,
    quantize_config=None,
)
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
INFO - The layer lm_head is not quantized.
INFO:auto_gptq.modeling._base:The layer lm_head is not quantized.


### Prompt Settings


Any given prompt will be connected with the default system promt.

In [ ]:
DEFAULT_SYSTEM_PROMPT = """
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
""".strip()


def generate_prompt(prompt: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT) -> str:
    return f"""
[INST] <>
{system_prompt}
<>

{prompt} [/INST]
""".strip()

In [ ]:
SYSTEM_PROMPT = "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer."

template = generate_prompt(
    """
{context}

Question: {question}
""",
    system_prompt=SYSTEM_PROMPT,
)
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

### Text Pipeline

In [ ]:
text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    temperature=0,
    top_p=0.95,
    repetition_penalty=1.15,
    streamer=streamer, # Ask this
)

    PyTorch 2.0.1+cu118 with CUDA 1108 (you have 2.2.1+cu121)
    Python  3.10.11 (you have 3.10.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details
The model 'LlamaGPTQForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'Marian

In [ ]:
llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0})

In [ ]:
def isClaimInDir(claimN, dir) -> bool:
  if os.path.isdir(dir + '/' + str(claimN)):
    return True
  else:
    return False

def retrieveFood(claimN) -> str:
  if not isClaimInDir(str(claimN), 'pdfs'):
    return '~'
  docs = []
  for k in os.listdir('pdfs/'+str(claimN)):
    if k.split('.')[1] == 'pdf':
      docs.append(k)
  for k in docs:
    reader = PdfReader('pdfs/'+str(claimN)+'/'+k)
    text = reader.pages[0].extract_text().replace('\n','')
    claim_match = re.search(r'the substantiation of health claims related to (.*?) and', text)
    if claim_match:
      claim_text = claim_match.group(1)
      print(f'Constituent Found: {claim_text}')
      return claim_text

def claimReport(claimN):
  claimN = str(claimN)

  #Preconditions
  if not isClaimInDir(claimN, 'pdfs'):
    return 'Claim Not Found in PDFS'
  if isClaimInDir(claimN, 'db'):
    shutil.rmtree(f'db/{claimN}')

  #Find Food
  constituent = retrieveFood(claimN)
  if constituent == '~':
    return 'Could not identify constituent'

  #Load QA retrieval and embedding model
  loader = PyPDFDirectoryLoader("pdfs/" + claimN)
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
  texts = text_splitter.split_documents(loader.load())
  db = Chroma.from_documents(texts, HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-large", model_kwargs={"device": DEVICE}), persist_directory="db/"+claimN)
  qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
  )
  print('---------------------------------------------------------')
  result = qa_chain('Prompt: Considering the text, what are all the claimed effects proposed by the panel? Please answer specifically following the structure: {Constituent} contributes to {Action} of subject assuming the target population is {Target} based on the paragraph {Paragraph}. Repeat for all claimed effects assumed by the panel, knowing that the constituent is ' + constituent + ' and that {Action} is a verb. Hint: All the claimed effects are listed in the index as subparagraph 2.x under the second paragraph titled "Relevance of the claimed effect to human health." Be sure to check the index for a comprehensive list of claimed effects. Additional Instructions: Keep in mind that each claimed effect has its own subparagraph under the second paragraph titled "Relevance of the claimed effect to human health," which spans over multiple pages and ends with the scientific substantiation of the claimed effect. At the end, please copy all the previously mentioned paragraphs from the provided source document in full.')
  print('---------------------------------------------------------')
  # result_B = qa_chain('Considering the text,what are the other academic papers cited? They are mostly containing citations as "(author et al., publish year)". Give the exact paragraph provides the supporting evidences with citations. At the end, please copy all the previously mentioned paragraphs from the provided source document in full.')
  # result_B = qa_chain('Generate detailed responses for each claimed effect proposed by the panel regarding the constituent ' + constituent +'. Include the complete text from the document for each claimed effect.')
  print('---------------------------------------------------------')

  #print(docs)
  return [result,result_B]

In [ ]:
t = claimReport(1)

Constituent Found: alpha-linolenic acid
load INSTRUCTOR_Transformer
max_seq_length  512
---------------------------------------------------------
 Based on the provided text, the claimed effects proposed by the panel are:

{Constituent} contributes to {Action} of subject assuming the target population is {Target} based on the paragraph {Paragraph}.

1. Alpha-linolenic acid contributes to the maintenance of normal blood cholesterol concentrations assuming the target population is the general population based on paragraph 2.1.
2. Alpha-linolenic acid helps maintain a healthy blood pressure assuming the target population is the general population based on paragraph 2.2.

Please note that these claimed effects are listed in the index as subparagraphs 2.1 and 2.2 under the second paragraph titled "Relevance of the claimed effect to human health."

Here are the full paragraphs from the provided source document:

"2. Relevance of the claimed effect to human health

2.1. Maintenance of normal 